In [60]:
from collections import Counter, defaultdict, namedtuple
from functools import partial
import pandas as pd
import numpy as np
import math, random
import glob

Current exercise has the following steps:
1. create a DECODER model that will discriminate computer code and identify the language by the code features.
2. the input training and test files needs to be prepared.
3. Target Structure to feed into decoding model:
    - training set: [file ID, [set of features, known code])
    - testing set: [file ID, [set of features, known code])
3. the discriminating features of code are not yet defined. 
4. the files need to be processed for input.
5. Need to decide which classifier is useful.

Strategy and Initial observations:
1. Envisioning a set of data that includes a row number, a profile of regex distributions, and the code type. 
2. Need to determine how to read in a file name, capture the file extension and associate it with the regex text profile.
3. Immerse self in the TextBlob software. 
4. Need to understand the role of glob.glob.


Loading test data:
1. glob the test set of files: 1 -> 32
2. append the test.csv code label (answer key for code file 1 to 32)
   - There are 2-4 code assignments per code type in the test data.
   - Target training to test ratio should between 60/40 and 75/25.
   - Therefore, need a minimum of 16 training snippets per code to cover the minimum 25% (4test/16train).
3. initial train/testing will focus on one translater.
4. quick build - a test and a benchmark. [start with ONE]
   - build a test for Python.
   - build a training set for Python

Will start by getting control of the test data:

In [61]:
def read_snippets_test(num):
    snips=[]
    for i in range(1, num):
        filetest = glob.glob('test/{}'.format(i))
        for files in filetest:
            with open(files) as f:
                snips.append(f.read())
    return snips

In [125]:
sniptest = read_snippets_test(33)

Verified that all test files read into sniptest with
- len(sniptest)
- type(sniptest)
- sniptest[0] - various slices compared to original files.

In [129]:
sniptestnames = pd.read_csv('test.csv',sep=',',header=None)

In [130]:
type(sniptestnames)

pandas.core.frame.DataFrame

In [131]:
len(sniptestnames)

32

Appended the column with the test code examples to the code key file: 
Verified that the content was read into the dataframe:

In [137]:
sniptestnames[2] = sniptest

In [141]:
sniptestnames[-3:]

,0,1,2
29,30,php,class Application extends App {\n\t/**\n\t * @...
30,31,ocaml,type name = string\n\nlet compare_label label1...
31,32,ocaml,let search_compiler_libs () =\n prerr_endline...


In [142]:
sniptestnames

,0,1,2
0,1,clojure,"(defn cf-settings\n ""Setup settings for campf..."
1,2,clojure,(ns my-cli.core)\n\n(defn -main [& args]\n (p...
2,3,clojure,(extend-type String\n Person\n (first-name [...
3,4,clojure,(require '[overtone.live :as overtone])\n\n(de...
4,5,python,from pkgutil import iter_modules\nfrom subproc...
5,6,python,import re\nimport subprocess\n\ndef cmd_keymap...
6,7,python,class NoSuchService(Exception):\n def __ini...
7,8,python,from collections import namedtuple\nimport fun...
8,9,javascript,function errorHandler(context) {\n return fun...
9,10,javascript,"var _ = require('lodash'),\n fs = require('..."


It appears the sniptest data is a list of the code from the files.  The sniptestnames data is a DataFrame.

Want to merge the name file with the code in the same order.  

First try:  convert list of code to a DataFrame and then merge the name column.
Since cannot get control of the test data will attempt to prepare the Training data.
As a test, I can pull

In [99]:
def read_snippets_train(directory):
    filetrain = glob.glob('benchmarksgame-2014-08-31/benchmarksgame/bench/{}/*.go*'.format(directory))
    snips = []
    for files in filetrain:
        with open(files) as f:
            snips.append(f.read())
    return snips

In [100]:
initial = read_snippets_train('binarytrees')

In [101]:
type(initial)

list

In [102]:
len(initial)

8

In [103]:
import csv

sms_data = []
sms_results = []

with open('test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        sms_data.append(row[1])
        sms_results.append(row[0])

In [104]:
len(sms_data)

32

In [105]:
len(sms_results)

32

In [106]:
len(sniptest)

31

In [110]:
type(sniptest)

list

In [136]:
sniptest[31]

'let search_compiler_libs () =\n  prerr_endline "I: Searching for OCaml compiler libraries";\n  let stdlib = BaseEnv.var_get "standard_library" in\n  let ( / ) = Filename.concat in\n  try\n    List.find (fun path -> Sys.file_exists (path / "types.cmi") || Sys.file_exists (path / "typing" / "types.cmi")) [\n      stdlib;\n      stdlib / "compiler-libs";\n      stdlib / "compiler-lib";\n      stdlib / ".." / "compiler-libs";\n      stdlib / ".." / "compiler-lib";\n    ]\n  with Not_found ->\n    prerr_endline "E: Cannot find compiler libraries! See the README for details.";\n    exit 1'